# Aim

Produce an RSS Scraper/Parser to convert news stories or articles to the Full Fact Haystack format.  

## Idea

* `feedparser` + `BeautifulSoup4` + `lxml` = Success!

## Findings

### RSS feeds can be non tivial

* https://www.theguardian.com/politics/series/the-politics-sketch/rss
* https://www.theguardian.com/politics/rss
* http://feeds.bbci.co.uk/news/politics/rss.xml

###  Some outlets only allow subscription via email newsletters

* http://www.politics.co.uk


### Other Challenges ( and TODOs)

* Need to read HTML article page and split content into sentences.
* Exporting to the Haystack XMl format
* Caching/storing articles already downloaded
* Scheduling the process and holding options as a config 

## Design/Plan

Store all the feed data in a tinyDB, only update information if it has changed?

What could change?

Title and content. How do we know its the same story? Best to capture everything? Do we then version the articles?

How do we identify an article?

* URL is good, combine with versioning?

**Use etag and Last modified to only get latest feeds**

Process each feed with:






# Minimal Example

In [2]:
import sys
sys.path.append('../package/src/')

In [3]:
import feedparser

In [3]:
bbc_url = "http://feeds.bbci.co.uk/news/politics/rss.xml"
ft_url = "http://feeds2.feedburner.com/ft/westminster"
gaurdian_url = "https://www.theguardian.com/politics/rss"

data_sources = [bbc_url, ft_url, gaurdian_url]

In [4]:
bbc_feed = feedparser.parse(bbc_url)

ft_feed = feedparser.parse(ft_url)

g_feed = feedparser.parse(gaurdian_url)

In [30]:
bbc_feed.entries[0].keys()

dict_keys(['title', 'title_detail', 'summary', 'summary_detail', 'links', 'link', 'id', 'guidislink', 'published', 'published_parsed', 'media_thumbnail', 'href'])

In [38]:
bbc_feed.entries[1].title_detail

{'base': 'http://feeds.bbci.co.uk/news/politics/rss.xml',
 'language': None,
 'type': 'text/plain',
 'value': 'Jeremy Corbyn on whether the UK will definitely leave the EU'}

In [28]:
g_feed.entries[0].keys()

dict_keys(['title', 'title_detail', 'links', 'link', 'summary', 'summary_detail', 'tags', 'published', 'published_parsed', 'id', 'guidislink', 'media_content', 'media_credit', 'credit', 'authors', 'author', 'author_detail', 'updated', 'updated_parsed'])

In [37]:
g_feed.entries[1].title_detail

{'base': 'https://www.theguardian.com/politics/rss',
 'language': None,
 'type': 'text/plain',
 'value': "Tory MPs plot to water down Theresa May's energy price cap pledge"}

In [29]:
ft_feed.entries[0].keys()

dict_keys(['title', 'title_detail', 'links', 'link', 'comments', 'published', 'published_parsed', 'authors', 'author', 'author_detail', 'tags', 'id', 'guidislink', 'summary', 'summary_detail', 'wfw_commentrss', 'slash_comments'])

In [ ]:
entry = bbc_feed.entries[0]

In [43]:
ft_feed.entries[0].wfw_commentrss

'http://blogs.ft.com/westminster/liveblogs/2017-04-18-2/feed/'

In [31]:
data = {
    'feed_id': 'foo',
    'title': entry.title, 
    'id': entry.id, 
    'link': entry.link, 
    'published': entry.published,
    'published_parsed': entry.published_parsed, 
    'authors': entry.get('authors'), 
    'tags': entry.get('tags')
}

NameError: name 'entry' is not defined

In [1]:
from storage import serialization
from tinydb import TinyDB, Query

db = TinyDB('db.json', storage=serialization)

import source_data
from source_data import fetch_new_entries

# sources are dictionaries with 
data_sources =  [
    {
        'name': 'bbc', 
        'url': "http://feeds.bbci.co.uk/news/politics/rss.xml", 
    }, 
    {
        'name': 'ft', 
        'url': "http://feeds2.feedburner.com/ft/westminster", 
    }, 
    {
        'name': 'gaurdian', 
        'url': "https://www.theguardian.com/politics/rss", 
    }    
]

result = db.table('meta').get(Query().name == 'ft')

ents = fetch_new_entries(source=data_sources[2], db=db)


ModuleNotFoundError: No module named 'storage'

In [1]:
import sys
sys.path.append('../package/src/')

In [2]:
import main

In [3]:
main.main(1)

TypeError: sequence item 0: expected str instance, spacy.tokens.token.Token found

In [4]:
%debug

> /Users/cmusselle/Mango/Workspace/rss-miner/package/src/main.py(61)main()
     59                 with open(filename, 'w') as f:
     60                     data = '\n-----\n'.join(sentences)
---> 61                     f.write(data)
     62 
     63                 time.sleep(0.1)

ipdb> sss = [x for x in sentences]
ipdb> sss
[]
ipdb> paragraphs
['Labour and the Lib Dems are both making multibillion pound funding pledges to tackle school cash shortages in England.', 'Labour says it would raise corporation tax to spend £4.8bn on keeping up with rising costs, and £335m so no schools lost out from a funding reorganisation.', 'The Lib Dems are promising £7bn to protect per-pupil funding levels.', "The Conservatives said school spending had reached record levels and attacked the two parties' spending plans.", 'Headteachers have been warning about job losses caused by cash shortages, with the National Audit Office estimating schools will have to cut £3bn from budgets by 2019-20 to meet ris

In [12]:
paragraphs = ['Labour and the Lib Dems are both making multibillion pounds. Funding pledges to tackle school cash shortages in England.', 'Labour says it would raise corporation tax to spend £4.8bn on keeping up with rising costs, and £335m so no schools lost out from a funding reorganisation.', 'The Lib Dems are promising £7bn to protect per-pupil funding levels.', "The Conservatives said school spending had reached record levels and attacked the two parties' spending plans.", 'Headteachers have been warning about job losses caused by cash shortages, with the National Audit Office estimating schools will have to cut £3bn from budgets by 2019-20 to meet rising cost pressures.', 'A new funding formula will also see some schools lose money, although others will gain.', 'On Monday, the Institute of Fiscal Studies said protecting schools from real-terms cuts in England would mean spending the equivalent of an extra penny on the basic rate of income tax.', 'Labour leader Jeremy Corbyn and shadow education secretary Angela Rayner are promising a real-terms funding increase in schools and "free lifelong education" in colleges, including restoring the Educational Maintenance Allowance.', 'This would be paid for by increasing corporation tax from the current 19% to 26% by 2021, Labour said.', '"We will invest in schools and in our young people, ensuring no primary pupils go hungry during the day, reducing class sizes so children can learn and teachers can teach, and restoring the maintenance allowance and grants for students in both further and higher education," Ms Rayner said.', 'The Lib Dems claimed their plans went further than Labour\'s and said details of how they would funded would be provided "in due course" in the party\'s manifesto.', 'The party said it would also protect further education per-pupil funding in real terms.', 'Lib Dem education spokeswoman Sarah Olney said: "While funding per pupil is set to see the biggest cuts in a generation, billions of pounds are being spent on divisive plans to expand grammars and free schools.', '"This extra £7 billion of funding would ensure no school and no child loses out."', 'The Conservatives said Labour had already committed the corporation tax rise to other projects.', 'Chief Secretary to the Treasury David Gauke said Labour\'s announcement was "just made-up promises on the back of nonsensical spending plans".', 'He added: "The Lib Dems are no better and won\'t even tell people about the tax rises they would bring in."', 'As education is a devolved matter, the pledges relate to England only, and the amount Scotland, Wales and Northern Ireland receives would be determined by the funding formula.']

import spacy
nlp = spacy.load('en')

docs = [nlp(p) for p in paragraphs]


In [15]:
list(docs[0].sents)

[Labour and the Lib Dems are both making multibillion pounds.,
 Funding pledges to tackle school cash shortages in England.]

In [18]:
from utils import flatten_nested_list

In [19]:
def paragraphs_to_sentences(paragraphs):
    """Return list of individual sentences from paragraphs"""

    import spacy
    nlp = spacy.load('en')

    docs = [nlp(p) for p in paragraphs]

    sents = [list(d.sents) for d in docs]

    all_sents = flatten_nested_list(sents)

    return all_sents


In [20]:
s = paragraphs_to_sentences(paragraphs)

In [23]:
s

<generator object flatten_nested_list at 0x1634cf8e0>

In [5]:
%ls articles

In [6]:
articles

NameError: name 'articles' is not defined

In [14]:
main(1)

FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?